In [9]:
import pandas as pd 
import joblib
import numpy as np
import torch
import torch.nn.functional as F 
import torch.nn as nn
from sklearn.metrics import r2_score
import time
import transformers
from torch.utils.data import Dataset

In [163]:
x=torch.randn(1000,100)

In [164]:
window=5
n_input=100
n_output=1

In [165]:
class short_window_dataset(Dataset):
    def __init__(self, _x, window):
        self.window = window
        self.n = _x.shape[0] - window + 1
        self._x = _x

    def __getitem__(self, idx):
        idx += self.window
        start = idx - self.window
        end = idx
        return self._x[start:end,:]

    def __len__(self):
        return self.n

class long_window_dataset(Dataset):
    def __init__(self, _x, window):
        self.window = window
        self.n = _x.shape[0] - window + 1
        self._x = _x

    def __getitem__(self, idx):
        idx += self.window
        end = idx
        return self._x[:end, :]

    def __len__(self):
        return self.n

In [166]:
datasetShort=short_window_dataset(x,window)
data_iter_short=torch.utils.data.DataLoader(datasetShort,1,shuffle=False,num_workers=10,pin_memory=True)
datasetLong=long_window_dataset(x,window)
data_iter_Long=torch.utils.data.DataLoader(datasetLong,1,shuffle=False,num_workers=10,pin_memory=True)

In [167]:
class RNNModel(nn.Module):
    def __init__(self, n_input,n_output):
        super(RNNModel,self).__init__()
        self.rnn=nn.GRU(n_input,n_output,batch_first=True)
    def forward(self,x):
        out, _ =self.rnn(x,None)
        out=out[:,-1,:]
        return out

In [168]:
device="cuda:2"
net=RNNModel(n_input,n_output).to(device)
for param in net.parameters():
    nn.init.normal_(param,mean=0,std=0.001)

In [169]:
listLong=[]
for x in data_iter_Long:
    listLong.append(((net(x.to(device)).cpu().detach().numpy()[0,0])))

In [170]:
listShort=[]
for x in data_iter_short:
    listShort.append(((net(x.to(device)).cpu().detach().numpy()[0,0])))

In [171]:
np.corrcoef(listLong,listShort)

array([[1.        , 0.99949864],
       [0.99949864, 1.        ]])